In [1]:
# get the data to add to the database
import stripe
from os import environ
import json
from datetime import datetime
import pytz
# authentication
api_secret_prod = environ['STRIPE_SECRET_KEY']

stripe.api_key = api_secret_prod

In [35]:
events = stripe.Event.list()

In [ ]:
events.auto_paging_iter()

In [82]:
def tryExcept(field_name):
    #--CUSTOMER ID & AMOUNTS----------
    try:
        value = event['data']['object'][field_name]
    except KeyError:
        value = None
    
    return value

In [83]:
def tryExceptAllFields(target_dict, field_name_list):
    field_values = {}
    for name in field_name_list:
        target_dict[name] = tryExcept(name)
    return target_dict

In [76]:
event_data={}
field_name_list=['customer','amount']
field_values = tryExceptAllFields(field_name_list)

customer
amount


In [77]:
field_values

{'customer': None, 'amount': None}

In [116]:
def getStripeEvents():
    events_list = []
    i=0
    events = stripe.Event.list(type='customer.updated')
    for event in events.auto_paging_iter():
        i+=1
        if event['object']=='event':
            event_json = json.loads(json.dumps(event))
            event_data={}
            event_data['stripe_event_id']=event_json['id']
            event_data['event_type']=event_json['type']
            
            ##--TIMESTAMP----------
            createdate_epoch = event['created']
            event_data['event_datetime_utc_epoch']=createdate_epoch
            # get time in tz
            tz_utc = pytz.timezone('UTC')
            event_data['event_datetime_utc'] = datetime.fromtimestamp(createdate_epoch, tz_utc).strftime('%Y-%m-%d %H:%M:%S')
            # get time in Central
            tz = pytz.timezone('America/Chicago')
            event_data['event_datetime'] = datetime.fromtimestamp(createdate_epoch, tz).strftime('%Y-%m-%d %H:%M:%S')
            
            #--CUSTOMER ID & AMOUNTS----------

            field_name_list = ['customer','amount_due','amount_paid','amount_remaining','amount','amount_captured','amount_refunded']            
            event_data = tryExceptAllFields(event_data, field_name_list)
           
            events_list.append(event_data)
        
    return events_list

In [ ]:
[x for x in events_list if x['stripe_event_id'] in ans]

In [43]:
invoice_events = [d for d in events_list if 'invoice.payment_succeeded' in d['event_type']]

In [56]:
events = stripe.Event.list(type='charge.succeeded')

In [97]:
#can do this with numpy.setdiff1d but I don't trust cPanel with numpy
def setdiff_sorted(array1,array2,assume_unique=False):
    ans = np.setdiff1d(array1,array2,assume_unique).tolist()
    if assume_unique:
        return sorted(ans)
    return ans

main_list = setdiff_sorted(list_b,list_a)

In [98]:
main_list

['f', 'g', 'h']

In [140]:
 
'''Use this to update MySQL table with all Stripe events'''    
def tryExcept(event,field_name):
    '''
    returns a value for a field in a Stripe event, or None if not available
    '''
    try:
        value = event['data']['object'][field_name]
    except KeyError:
        value = None
    
    return value

def fillStripeEventData(event, target_dict, field_name_list):
    '''
    populates event data (using try/except logic) for all fields
    '''
    field_values = {}
    for name in field_name_list:
        target_dict[name] = tryExcept(event,name)
    return target_dict

def tryExcept(event,field_name
    
    if 'cus' in event['data']['object']['id']:
        try:
            value = event['data']['object']['id'] #this is where customer_id lives for "customer" event types
        except KeyError:
            value = None  
    else:
        try:
            value = event['data']['object'][field_name]
        except KeyError:
            value = None
    
    return value

In [141]:
for event in events.auto_paging_iter():
    i+=1
    if event['object']=='event':
        #print(event['data']['object'])
        print(tryExcept(event,'customer'))

cus_IkR19D09w3YKcQ
cus_IlIgXV7d7LPkgX
cus_IlIgXV7d7LPkgX
cus_IkR19D09w3YKcQ
cus_IkR19D09w3YKcQ
cus_IjvViROhk7zi67
cus_IjvViROhk7zi67
cus_IdubrPcAfY6mmM
cus_IdubrPcAfY6mmM
cus_IbVq5lZRc7ElGJ
cus_IbVq5lZRc7ElGJ


In [125]:
    events_list = []
    all_stripe_event_ids = []
    i=0
    
    #cPanel hosting is janky
    stripe.max_network_retries = 10
    
    events = stripe.Event.list(type='customer.updated')
    for event in events.auto_paging_iter():
        i+=1
        if event['object']=='event':
            
            event_json = json.loads(json.dumps(event))
            
            all_stripe_event_ids.append(event_json['id'])
            event_data={}
            event_data['stripe_event_id']=event_json['id']
            event_data['event_type']=event_json['type']
            
            ##--TIMESTAMP----------
            createdate_epoch = event['created']
            event_data['event_datetime_utc_epoch']=createdate_epoch
            # get time in tz
            tz_utc = pytz.timezone('UTC')
            event_data['event_datetime_utc'] = datetime.fromtimestamp(createdate_epoch, tz_utc).strftime('%Y-%m-%d %H:%M:%S')
            # get time in Central
            tz = pytz.timezone('America/Chicago')
            event_data['event_datetime'] = datetime.fromtimestamp(createdate_epoch, tz).strftime('%Y-%m-%d %H:%M:%S')
            
           #--CUSTOMER ID & AMOUNTS----------

            field_name_list = ['customer','amount_due','amount_paid','amount_remaining','amount','amount_captured','amount_refunded']            
            event_data = fillStripeEventData(event, event_data, field_name_list)
                
            events_list.append(event_data)
    
    #filter out any that are already in the database
    existing_event_ids = []
    
    event_ids_to_push_to_db = [event_id for event_id in all_stripe_event_ids if event_id not in existing_event_ids]

    event_info_to_push_to_db = [event for event in events_list if event['stripe_event_id'] in event_ids_to_push_to_db]
    
    return event_info_to_push_to_db

In [126]:
getNewStripeEvents()

[{'stripe_event_id': 'evt_1IAiClGun3mcXhIzQ3TXXiRW',
  'event_type': 'customer.updated',
  'event_datetime_utc_epoch': 1610916635,
  'event_datetime_utc': '2021-01-17 20:50:35',
  'event_datetime': '2021-01-17 14:50:35',
  'customer': None,
  'amount_due': None,
  'amount_paid': None,
  'amount_remaining': None,
  'amount': None,
  'amount_captured': None,
  'amount_refunded': None},
 {'stripe_event_id': 'evt_1IAKFQGun3mcXhIzojIl9E8G',
  'event_type': 'customer.updated',
  'event_datetime_utc_epoch': 1610824543,
  'event_datetime_utc': '2021-01-16 19:15:43',
  'event_datetime': '2021-01-16 13:15:43',
  'customer': None,
  'amount_due': None,
  'amount_paid': None,
  'amount_remaining': None,
  'amount': None,
  'amount_captured': None,
  'amount_refunded': None},
 {'stripe_event_id': 'evt_1IAKDlGun3mcXhIzyz76eADU',
  'event_type': 'customer.updated',
  'event_datetime_utc_epoch': 1610824440,
  'event_datetime_utc': '2021-01-16 19:14:00',
  'event_datetime': '2021-01-16 13:14:00',
  'cu